In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa
import pandera as pa

# Local libraries
import pudl
import pudl.constants as pc
import pudl.validate as pv

In [4]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [5]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine) # freq='monthly'/'annual' (maybe other abr.)

In [6]:
bf_eia923 = pudl_out.bf_eia923()

In [7]:
bf_eia923.fuel_type_code.unique()

array(['BIT', 'NG', 'AB', 'DFO', 'WDS', 'LIG', 'SUB', 'OTH', 'WO', 'LFG',
       'PC', 'SC', 'RFO', 'OBG', 'OBL', 'TDF', 'WC', 'OBS', 'PG', 'OG',
       'KER', 'MSB', 'MSN', 'SLW', 'WH', 'JF', 'BLQ', 'BFG', 'WDL', 'PUR',
       'SGC', 'RC', 'SGP', 'ANT'], dtype=object)

In [8]:
df = pd.DataFrame({
    "report_year": [2010, 2011, 2012, 2010, 2011, 2012],
    "plant_id_eia": [1, 1, 1, 2, 2, 2],
    "fuel_consumed_units": [5668.0, 8925.0, 6665.0, 623561.0, 675227.0, 574865.0],
    "fuel_mmbtu_per_unit": [22.496, 22.710, 22.798, 1.032, 1.032, 1.032],
    "fuel_type_code": ['BIT', 'BIT', 'BIT', 'NG', 'NG', 'NG'],
})

In [28]:
df.sort_values('fuel_mmbtu_per_unit')
df1=df[df['fuel_type_code']=='BIT']
sn = df1.fuel_consumed_units.cumsum()
sn

0     5668.0
1    14593.0
2    21258.0
Name: fuel_consumed_units, dtype: float64

In [37]:
pn = (sn - 0.5 * df1.fuel_consumed_units) / sn.iloc[-1]

import numpy as np
np.interp(0.5, pn, df1.fuel_mmbtu_per_unit)
pn

0    0.133315
1    0.476550
2    0.843235
Name: fuel_consumed_units, dtype: float64

In [31]:
df1

,report_year,plant_id_eia,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code
0,2010,1,5668.0,22.496,BIT
1,2011,1,8925.0,22.710,BIT
2,2012,1,6665.0,22.798,BIT


In [59]:
# Pre-validate function
def weighted_col(data_col, weight_col):
    Sn = weight_col.cumsum()  # noqa: N806
    # This conditional is necessary because sometimes new columns get
    # added to the EIA data, and so they won't show up in prior years.
    if len(Sn) > 0:
        Pn = (Sn - 0.5 * weight_col) / Sn.iloc[-1]  # noqa: N806
    return Pn


# PRE VALIDATE
def pre_validate_bf_923(df):
    df = (
        df.assign(
            Pn=lambda x: weighted_col(
                x.fuel_mmbtu_per_unit, 
                x.fuel_consumed_units)
        )
    )
    return df

In [79]:
df2 = pre_validate_bf_923(df1)
np.interp(0.95, df2.Pn, df2.fuel_mmbtu_per_unit)


22.798

,report_year,plant_id_eia,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code
0,2010,1,5668.0,22.496,BIT
1,2011,1,8925.0,22.710,BIT
2,2012,1,6665.0,22.798,BIT


In [133]:
# validate function

def test_bounds(df, data_col, fuel, high, low):
        
    df = df.query(f"fuel_type_code=='{fuel}'")
    within_bounds = False
    
    Sn = weight_col.cumsum()
    if len(Sn) > 0:
        Pn = (Sn - 0.5 * weight_col) / Sn.iloc[-1] 
    else:
        Pn = np.nan
        
    high_quant = np.interp(0.95, df.Pn, df[f'{data_col}'])
    if high_quant > high:
        raise ValueError(
            f"95% quantile ({high_quant}) "
            f"is above upper bound ({high}) "
            f"in validation entitled {title}"
        )
    else: within_bounds = True
    low_quant = np.interp(0.05, df.Pn, df[f'{data_col}'])
    if low_quant < low:
        raise ValueError(
            f"5% quantile ({low_quant}) "
            f"is below lower bound ({low}) "
            f"in validation entitled {title}"
        )
    else: within_bounds = True
    
    return within_bounds

In [134]:
schema_bf_923 = pa.DataFrameSchema({
    "fuel_mmbtu_per_unit": pa.Column(pa.Float, nullable=True)
}, checks=[
    #pa.Check(lambda df: (
    #    test_bounds(df, df.fuel_mmbtu_per_unit, fuel='BIT',high=26.5, low=20.5))),
    pa.Check(lambda df: (
        test_bounds(df, 'fuel_mmbtu_per_unit', fuel='NG', high=1.036, low=1.018)))
])

In [135]:
pre_validated_bf_923 = pre_validate_bf_923(df)
schema_bf_923.validate(pre_validated_bf_923)


,report_year,plant_id_eia,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code,Pn
0,2010,1,5668.0,22.496,BIT,0.001496
1,2011,1,8925.0,22.710,BIT,0.005346
2,2012,1,6665.0,22.798,BIT,0.009460
3,2010,2,623561.0,1.032,NG,0.175754
4,2011,2,675227.0,1.032,NG,0.518458
5,2012,2,574865.0,1.032,NG,0.848313


In [30]:
def test(df, col):
    return df[col] < 100000000000

In [43]:
def vs_bounds(df, data_col, weight_col, query="", title="",
              low_q=False, low_bound=False, hi_q=False, hi_bound=False):
    """Test a distribution against an upper bound, lower bound, or both."""
    # These assignments allow 0.0 to be used as a bound...
    low_bool = low_bound is not False
    hi_bool = hi_bound is not False

    if bool(low_q) ^ low_bool and low_q != 0:
        raise ValueError(
            f"You must supply both a lower quantile and lower bound, "
            f"or neither. Got: low_q={low_q}, low_bound={low_bound} "
            f"for validation entitled {title}"
        )
    if bool(hi_q) ^ hi_bool:
        raise ValueError(
            f"You must supply both a lower quantile and lower bound, "
            f"or neither. Got: low_q={hi_q}, low_bound={hi_bound} "
            f"for validation entitled {title}"
        )

    if query != "":
        df = df.copy().query(query)
    if title != "":
        logger.info(title)
    if weight_col is None or weight_col == "":
        df["ones"] = 1.0
        weight_col = "ones"
    if low_q >= 0 and low_bool:
        low_test = pv.weighted_quantile(df[data_col], df[weight_col], low_q)
        logger.info(f"{data_col} ({low_q:.0%}): "
                    f"{low_test:.6} >= {low_bound:.6}")
        if low_test < low_bound:
            raise ValueError(
                f"{low_q:.0%} quantile ({low_test}) "
                f"is below lower bound ({low_bound}) "
                f"in validation entitled {title}"
            )
        else:
            low_bool= low_test > low_bound
    if hi_q <= 1 and hi_bool:
        hi_test = pv.weighted_quantile(df[data_col], df[weight_col], hi_q)
        logger.info(f"{data_col} ({hi_q:.0%}): {hi_test:.6} <= {hi_bound:.6}")
        if hi_test > hi_bound:
            raise ValueError(
                f"{hi_q:.0%} quantile ({hi_test}) "
                f"is above upper bound ({hi_bound}) "
                f"in validation entitled {title}"
            )
        else:
            hi_bool=hi_test < hi_bound
            
    return low_bool & hi_bool

In [44]:
schema = pa.DataFrameSchema({
    "fuel_mmbtu_per_unit": pa.Column(pa.Float, nullable=True)
}, checks=[
    pa.Check(lambda df: test(df, 'fuel_mmbtu_per_unit')),        
    pa.Check(lambda df: vs_bounds(
        df=df,
        data_col="fuel_mmbtu_per_unit",
        weight_col="fuel_consumed_units",
        query="fuel_type_code=='BIT'",
        title="Bituminous coal heat content (middle)",
        low_q=0.50,
        low_bound=20.5,
        hi_q=0.50,
        hi_bound=26.5,
    ), ignore_na=True)
])

In [45]:
schema.validate(bf_eia923)

Bituminous coal heat content (middle)
fuel_mmbtu_per_unit (50%): 23.59 >= 20.5
fuel_mmbtu_per_unit (50%): 23.59 <= 26.5


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,boiler_id,ash_content_pct,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code,fuel_type_code_pudl,sulfur_content_pct,total_heat_content_mmbtu
0,2011-01-01,3,32,Barry,195,18,Alabama Power Co,1,5.5,5668.0,22.496,BIT,coal,0.44,127507.328
1,2011-02-01,3,32,Barry,195,18,Alabama Power Co,1,6.1,8925.0,22.710,BIT,coal,0.59,202686.750
2,2011-03-01,3,32,Barry,195,18,Alabama Power Co,1,6.6,6665.0,22.798,BIT,coal,0.48,151948.670
3,2011-04-01,3,32,Barry,195,18,Alabama Power Co,1,5.9,3919.0,22.796,BIT,coal,0.50,89337.524
4,2011-05-01,3,32,Barry,195,18,Alabama Power Co,1,6.9,7570.0,22.712,BIT,coal,0.67,171929.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864559,2018-08-01,60903,7784,Salem Harbor Station NGCC,59928,4608,Footprint Salem Harbor Development LP,4,0.0,623561.0,1.032,NG,gas,0.00,643514.952
864560,2018-09-01,60903,7784,Salem Harbor Station NGCC,59928,4608,Footprint Salem Harbor Development LP,4,0.0,675227.0,1.032,NG,gas,0.00,696834.264
864561,2018-10-01,60903,7784,Salem Harbor Station NGCC,59928,4608,Footprint Salem Harbor Development LP,4,0.0,574865.0,1.032,NG,gas,0.00,593260.680
864562,2018-11-01,60903,7784,Salem Harbor Station NGCC,59928,4608,Footprint Salem Harbor Development LP,4,0.0,14413.0,1.028,NG,gas,0.00,14816.564
